In [1]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException
import datetime
import pytz
import openpyxl
import pandas as pd
import numpy as np
import csv
import glob
import pyodbc
import os 
import json
from io import BytesIO
import io
import platform
from function.PyToSp import *
from tqdm import tqdm
from urllib.parse import quote_plus
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, event,text
import pyodbc
import requests
import inspect
from validate import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from urllib.parse import quote_plus
import msal
from itertools import chain
from send_email import *
from Connection import *


In [2]:
def ConnectSharePoint(url_hub):
    header_Hub = f"share_point_{url_hub.split('/')[2].replace('-','')}"
    config_Hub = read_config_json(config_path, header_Hub)
    Hub = SharePoint(config_Hub)
    #Hub.check_connect()
    return Hub

def ConnectAzureSQLServer(): 
    f = open ('database_information.json', "r") #Database information file, can change information depending on the time
    qq = json.loads(f.read())
    f.close()
    ini_cnt_str ='Driver={driver_str};Server=tcp:{servername},1433;database={database};Uid={username};Pwd={password};Encrypt=yes;Authentication=ActiveDirectoryPassword;Connection Timeout=30;'.format(**qq)
    quoted = quote_plus(ini_cnt_str)
    cnt_str = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
    engine = create_engine(cnt_str)

        #Test Connection
    try:
        conn = engine.connect()
        result = conn.execute(text("SELECT 1"))
        print("CONNECTION SUCESSFUL!")
    except Exception as e:
        print("CONNECTION FAILED:",str(e))
    return cnt_str
cnt_str = ConnectAzureSQLServer()

CONNECTION SUCESSFUL!


In [3]:
def check_data_lenght(Sector, url_file):
    excel_data = pd.DataFrame()
    excel_data, file_name, sector = get_data(url_hub, url_file)
    data_type = pd.read_excel('doc/DataType.xlsx', sheet_name= Sector)
    # Iterate over the columns in the DataFrame
    for column in excel_data.columns:
        lenght_data = excel_data[column].astype(str).str.len().max()
        for col in data_type['column_name']:
            if col == column: 
                index = data_type[data_type['column_name'] == col].index[0]
                max_length = data_type.loc[index, 'max_length']
                if max_length != 'nan':
                    if lenght_data > max_length:
                        max_length = int(max_length)
                        print(f'{col} có độ dài tối đa là: ', max_length)
                        print(f'{column}: ', lenght_data)


In [4]:
def compare_Template(Sector,url_hub, file_url):
    # Read the first line of each file into separate dataframes
    data, file_name, sector = get_data(url_hub, file_url)

    df1 = pd.DataFrame(data)
    df2 = pd.read_excel(f'doc/Template/Template_{Sector}.xlsx', header=None, nrows=1)

    # Compare the values of the first lines
    diff_values = {}
    for i in range(len(df1.columns)):
        if df1.columns[i] != df2.iloc[0, i]:
            diff_values[i] = (df1.columns[i], df2.iloc[0, i])
    return diff_values 


In [5]:
def GetProvinceIS(provinces, list_folder):  
        selected_provinces = []
        province_list = []
        for i in list_folder:
            province_list.append(i.split('/')[7])
        for input_province in provinces:
            input_province = input_province.strip().lower()
            if input_province == "-1":
                break

            matched_provinces = [province for province in province_list if input_province in province.lower()]
            if matched_provinces:
                selected_provinces.extend(matched_provinces)
            else:
                print(f"Không tìm thấy tỉnh chứa từ khóa '{input_province}'.")
        return selected_provinces 
def GetProvince(input_string, list_folder ,selected_provinces):  
        input_list = input_string.split(',')
        province_list = []
        for i in list_folder:
            province_list.append(i.split('/')[7])
        for input_province in input_list:
            input_province = input_province.strip().lower()
            if input_province == "-1":
                break

            matched_provinces = [province for province in province_list if input_province in province.lower()]
            if matched_provinces:
                selected_provinces.extend(matched_provinces)
            else:
                print(f"Không tìm thấy tỉnh chứa từ khóa '{input_province}'.")
        return selected_provinces 

In [6]:
def getListImportFile(list_url):
    url = []
    for i in list_url:
        sector = i.split('/')[-1].split('_')[0].upper()
        if sector not in ('x'):
            url.append(i)
        else:
            pass
    return url
def findFolderWithProvince(list_folder, selected_provinces  ,Year, Quarter):
        # Tìm các folder chứa từ khóa
        selected_province_folders = [folder for folder in list_folder if any(re.search(keyword, folder, re.IGNORECASE) for keyword in selected_provinces)]
        selected_year_quarter = []
        for folder in selected_province_folders:
            folder_sub2 = folder + '/' + Year + '/' + Quarter
            selected_year_quarter.append(folder_sub2)
        # In danh sách các folder đã tìm thấy
        return selected_year_quarter
def findFolder(list_folder,selected_provinces , url_hub, year, quarter):
        df_summ_file = pd.DataFrame({'Name':[],'ServerRelativeUrl':[], 'TimeLastModified':[], 'ModTime':[], 'Modified_by_ID':[]})
        for i in findFolderWithProvince(list_folder, selected_provinces, year, quarter):
            if i.split('/')[7] in selected_provinces and i.split('/')[8] in year and i.split('/')[9] in quarter:
                df_summ_file = pd.concat([df_summ_file, ConnectSharePoint(url_hub).get_content_url(i)])
        list_url = df_summ_file['ServerRelativeUrl'].to_list()
        #History file
        df_query=pd.DataFrame(df_summ_file).reset_index(drop=True)
        df_summ_file = df_summ_file.reset_index(drop=True)
        return list_url , df_summ_file
def getFile(list_folder, selected_provinces, url_hub):
    # Input provinces
    input_string = input("Nhập tên các tỉnh bạn muốn chọn, cách nhau bằng dấu phẩy (nhập -1 để thoát): ")
    # Input Năm và Quý
    quarter_year = input("Nhập chuỗi năm và quý (ví dụ: 2023Q1): ")
    Year, Quarter = quarter_year[:4], quarter_year[4:]

    # In danh sách tỉnh đã chọn, năm và quý
    print("Các tỉnh đã chọn:")
    for province in GetProvince(input_string, list_folder ,selected_provinces):
        print(colored(province,'yellow'))
    print(colored("Year:{}, Quarter:{}".format(Year,Quarter),'yellow'))
    findFolderWithProvince(list_folder, selected_provinces,Year,Quarter)
    list_url , df_summ_file = findFolder(list_folder, selected_provinces, url_hub, Year, Quarter)
    list_url = getListImportFile(list_url)
    return list_url, df_summ_file
# def getFileMainSector(Subsector ,list_folder, selected_provinces, url_hub):
#     url = []
#     # Input provinces
#     input_string = input("Nhập tên các tỉnh bạn muốn chọn, cách nhau bằng dấu phẩy (nhập -1 để thoát): ")
#     # Input Năm và Quý
#     quarter_year = input("Nhập chuỗi năm và quý (ví dụ: 2023Q1): ")
#     Year, Quarter = quarter_year[:4], quarter_year[4:]
#     for province in GetProvince(input_string, list_folder ,selected_provinces):
#         print(colored(province,'yellow'))
#     print(colored("Year:{}, Quarter:{}".format(Year,Quarter),'yellow'))
#     findFolderWithProvince(list_folder, selected_provinces,Year,Quarter)
#     list_url , df_summ_file = findFolder(list_folder, selected_provinces, url_hub, Year, Quarter)
#     for i in list_url:
#             sector = i.split('/')[-1].split('_')[0].upper()
#             if sector in (Subsector):#Chọn sector cần import: RETAIL, OFFICE, SA, HOTEL, APT, VLTH
#                 url.append(i)
#             else:
#                 pass
#     return url, df_summ_file

In [7]:
def validateIP(url_hub, file_url,cnt_str, sp_object,df_summ_file, Hub):
    columns_that_need_unidecode=['Project_Name', 'Sub_Project_Name', 'Developer_Name'
                                , 'City', 'District', 'Target_Industry']
    engine = create_engine(cnt_str)
    #Create empty df for checking dictionary
    df_dict = pd.DataFrame(columns=['File_Name', 'Missing_Values', 'Flag'])
    df_flat_ip = pd.DataFrame()
    df_dup= pd.DataFrame()
    df_temp_flat_ip = pd.DataFrame() 
    #Create multi empty df for tracking autdit step
    name_sector = ['IP']
    name_sector = [x.lower() for x in name_sector]
    for i in name_sector:
        globals()['df_temp_flat_{}'.format(i)] = pd.DataFrame([])
        globals()['df_flat_{}'.format(i)] = pd.DataFrame([])
        globals()['df_new_key_{}'.format(i)] = pd.DataFrame([])  
    #-------------------------------------------------------
    '''Get data''' 
    print(f'Start Get Data {file_url}')
    data, file_name, sector = get_data(url_hub, file_url)
    print(file_name, sector)
    #-------------------------------------------------------
    data = check_date_key(file_url, data)#Check format date_key in flat file   
    data['Project_Name']= np.where(data['Project_Name'].isnull(), data['Sub_Project_Name'], data['Project_Name'])#Fill up project_name if its null
    data['Project_Sub_Type']= np.where(data['Project_Sub_Type'].isnull(), data['Project_Type'], data['Project_Sub_Type'])#Fill up Sub_Type if its null

    #Check duplicate sub_name
    print('Start Check duplicate sub_name...')
    data, df_dup = check_duplicate(data, 'Sub_Project_Name')
    print(colored('Check duplicate sub_name', 'yellow'))
    #-------------------------------------------------------
    '''Validation step'''
    print('Start validation step...')
    #Remove unfortmated values
    data = remove_unformated_character(data)
    #Remove unicode characters
    for i in columns_that_need_unidecode:
        data[i] = remove_unicode(data[i])
    #Check dictionary
    print('Start Check dictionary...')
    lst_dict = ['City', 'District', 'Status', 'Sub_Type', 'Type']
    lst_cls = ['City', 'District', 'Status', 'Project_Sub_Type', 'Project_Type']
    for i, j in zip(lst_cls, lst_dict):
        print(f'Start checking dictionary of [{i}:{j}]...')
        data, df_dict = check_dictionary(df_dict, file_name, data, i, j, sector, engine, sp_object)
    print(f'End Check dictionary with df_dict = {len(df_dict)} ...')
    
    if len(df_dict) == 0:
        print(colored('Validate succesfully','green'))
    else:
        #-------------------------------------------------------
        '''Import data process'''
        #Check project key
        data = Generate_Additional_Columns(data,df_summ_file, Hub,engine,file_url)
        processed_data, flag_key = check_project_key(file_url, data, sector, engine)
        df_temp_flat_ip = pd.concat([df_temp_flat_ip, data], axis=0)
        df_flat_ip = tracking_flat_file(df_temp_flat_ip, file_url)
#         if len(processed_data) != 0:
#             df_new_key_ip = check_new_key(df_new_key = df_new_key_ip, processed_data = processed_data, sector = sector)
    return df_dup, df_dict, df_flat_ip, df_temp_flat_ip

In [ ]:
df_dup= pd.DataFrame()
df_dict= pd.DataFrame()
df_flat_ip = pd.DataFrame()
df_temp_flat_ip= pd.DataFrame()
diff_values ={}
selected_provinces = [] 
while(True):
        Sector= input("Nhập Sector bạn muốn lấy data: ")
        url_hub=f'/sites/BIHub/Shared Documents/Advisory Data/{Sector}/Flat file'
        Hub= ConnectSharePoint(url_hub)
        sp_object = url_hub.split('/')[2].replace('-','')
        list_folder = Hub.get_content_url(url_hub, return_list_folder=True)
        if(Sector == 'Main Sector'):
            url, df_summ_file = getFile(list_folder, selected_provinces, url_hub)
            for i in url: 
                sector = i.split('/')[-1].split('_')[0].upper()
                diff_values = compare_Template(sector,url_hub, i)
                check_data_lenght(sector, i)
                df_dup, df_dict,df_flat_ip, df_temp_flat_ip= validateIP(url_hub, i,cnt_str, sp_object,df_summ_file, Hub)
        if (Sector == 'IP'):
            url, df_summ_file = getFile(list_folder, selected_provinces, url_hub)
            for i in url: 
                diff_values = compare_Template(Sector,url_hub, i)
                check_data_lenght(Sector, i)
                df_dup, df_dict,df_flat_ip, df_temp_flat_ip= validateIP(url_hub, i,cnt_str, sp_object,df_summ_file, Hub)

Nhập Sector bạn muốn lấy data: IP
Folder name: /sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Tien Giang Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Nam Dinh Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Vinh Phuc Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Hai Duong Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Binh Duong Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Quang Nam Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Ninh Binh Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Thai Nguyen Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Nghe An Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Dong Nai Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Quang Ngai Province
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Da Nang City
/sites/BIHub/Shared Documents/Advisory Data/IP

##### Check column name 

In [ ]:
print("Different values:")
for col, (val1, val2) in diff_values.items():
    print(f"Column {col+1}: {val1} != {val2}")

In [ ]:
df_flat_ip

##### Check with Sub_Project_Name
If there are 2 duplicate Sub_Project_Name, an error will be displayed

In [ ]:
df_dup

In [ ]:
df_dict